In [14]:
import json
import random

IS_DEBUG_MODE = False
DEBUG_SAMPLE_SIZE = 8

NUM_OF_DOCS = 10 
SPLIT_EXPERIMENT = False

DATASET_CONFIGS = [
    {
        "path": "/mnt/raid5/kangjh/Research/datasets/2wikimultihopqa/dev.json",
        "name": "2wikimultihopqa",
        "format": "json",
        "question_field": "question",
        "answer_field": "answer"
    },
    {
        "path": "/mnt/raid5/kangjh/Research/datasets/musique/validation.json",
        "name": "musique",
        "format": "json",
        "question_field": "question",
        "answer_field": "answer"
    },
    {
        "path": "/mnt/raid5/kangjh/Research/datasets/hotpotqa/validation.jsonl",
        "name": "hotpotqa",
        "format": "jsonl",
        "question_field": "question",
        "answer_field": "answer"
    }
]

OUTPUT_FILENAME = 'benchmark_records.jsonl'


def load_data(config):
    file_path = config["path"]
    file_format = config["format"]
    
    print(f"'{config['name']}' 데이터셋 로딩 중... ({file_path})")
    
    data_points = []
    try:
        with open(file_path, 'r', encoding='utf-8') as f:
            if file_format == 'json':
                data_points = json.load(f)
            elif file_format == 'jsonl':
                for line in f:
                    if line.strip():
                        data_points.append(json.loads(line))
            else:
                print(f"지원하지 않는 파일 형식입니다: {file_format}")
                return []
    except FileNotFoundError:
        print(f"[오류] 파일을 찾을 수 없습니다: {file_path}")
        return []
    except json.JSONDecodeError:
        print(f"[오류] JSON 파싱 중 오류가 발생했습니다: {file_path}")
        return []
        
    print(f"로딩 완료: 총 {len(data_points)}개 데이터")
    return data_points

def main():
    """데이터셋들을 읽어 하나의 JSONL 파일로 통합 생성합니다."""
    final_records = []

    for config in DATASET_CONFIGS:
        # 1. 데이터 로드
        datas = load_data(config)
        if not datas:
            continue

        # 2. 디버그 모드 시 샘플링
        if IS_DEBUG_MODE:
            if len(datas) > DEBUG_SAMPLE_SIZE:
                datas = random.sample(datas, DEBUG_SAMPLE_SIZE)
            print(f"디버그 모드: {len(datas)}개 샘플링 완료.")

        # 3. 레코드 생성
        # 이전 코드의 for문 구조를 그대로 활용하여 실험 파라미터를 적용합니다.
        for split in [SPLIT_EXPERIMENT]:
            for num_passages_one_retrieval in [NUM_OF_DOCS]:
                # split이 True일 경우 [6, 8]과 같은 리스트를, False일 경우 부모 루프의 값을 그대로 사용
                num_passages_options = ([6, 8] if split else [num_passages_one_retrieval])
                for num_passages_one_split_retrieval in num_passages_options:
                    
                    for data in datas:
                        question = data.get(config["question_field"], "")
                        answer = data.get(config["answer_field"], "")

                        # 데이터 필드가 없는 경우 건너뛰기
                        if not question or not answer:
                            continue

                        record = {
                            'dataset': config['name'],
                            'problem': question,
                            'golden_answers': answer,
                            'num_passages_one_split_retrieval': num_passages_one_split_retrieval,
                            'num_passages_one_retrieval': num_passages_one_retrieval,
                            'split': split,
                            'context': f"The question: {question}",
                            'split_querys': [],
                            "docs": [],
                            "state": "undo",
                            'resample_times': 0
                        }
                        final_records.append(record)
    
    print(f"\n총 {len(final_records)}개의 레코드가 생성되었습니다.")

    # 4. JSONL 파일로 저장
    with open(OUTPUT_FILENAME, 'w', encoding='utf-8') as f:
        for record in final_records:
            f.write(json.dumps(record, ensure_ascii=False) + '\n')
            
    print(f"성공적으로 '{OUTPUT_FILENAME}' 파일에 저장했습니다.")


if __name__ == '__main__':
    main()

'2wikimultihopqa' 데이터셋 로딩 중... (/mnt/raid5/kangjh/Research/datasets/2wikimultihopqa/dev.json)
로딩 완료: 총 12576개 데이터
'musique' 데이터셋 로딩 중... (/mnt/raid5/kangjh/Research/datasets/musique/validation.json)
로딩 완료: 총 2417개 데이터
'hotpotqa' 데이터셋 로딩 중... (/mnt/raid5/kangjh/Research/datasets/hotpotqa/validation.jsonl)
로딩 완료: 총 7405개 데이터

총 22398개의 레코드가 생성되었습니다.
성공적으로 'benchmark_records.jsonl' 파일에 저장했습니다.


In [4]:
file_path = '/mnt/raid5/kangjh/Research/datasets/musique/validation.json'

with open(file_path, 'r', encoding='utf-8') as f:
    data_points = json.load(f)

    

JSONDecodeError: Extra data: line 2 column 1 (char 10472)

In [5]:
import datasets

/home/kangjh/assignment/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [7]:
musique = datasets.load_dataset("dgslibisey/MuSiQue", split="validation")

In [8]:
musique

Dataset({
    features: ['id', 'paragraphs', 'question', 'question_decomposition', 'answer', 'answer_aliases', 'answerable'],
    num_rows: 2417
})

In [10]:

musique_dataset = datasets.load_dataset("dgslibisey/MuSiQue", split="validation")
print(f"✅ 데이터셋 로딩 완료! (총 {len(musique_dataset)}개 데이터)")

output_filename = "musique_validation.json"
data_list = [item for item in musique_dataset]
print(f"💡 데이터셋을 리스트 형태로 변환했습니다.")

print(f"'{output_filename}' 파일로 저장 중...")
with open(output_filename, 'w', encoding='utf-8') as f:
    json.dump(data_list, f, indent=4, ensure_ascii=False)

print(f"🎉 성공적으로 '{output_filename}' 파일에 저장했습니다.")

✅ 데이터셋 로딩 완료! (총 2417개 데이터)
💡 데이터셋을 리스트 형태로 변환했습니다.
'musique_validation.json' 파일로 저장 중...
🎉 성공적으로 'musique_validation.json' 파일에 저장했습니다.


In [11]:

musique_dataset = datasets.load_dataset("dgslibisey/MuSiQue", split="train")
print(f"✅ 데이터셋 로딩 완료! (총 {len(musique_dataset)}개 데이터)")

output_filename = "musique_train.json"
data_list = [item for item in musique_dataset]
print(f"💡 데이터셋을 리스트 형태로 변환했습니다.")

print(f"'{output_filename}' 파일로 저장 중...")
with open(output_filename, 'w', encoding='utf-8') as f:
    json.dump(data_list, f, indent=4, ensure_ascii=False)

print(f"🎉 성공적으로 '{output_filename}' 파일에 저장했습니다.")

✅ 데이터셋 로딩 완료! (총 19938개 데이터)
💡 데이터셋을 리스트 형태로 변환했습니다.
'musique_train.json' 파일로 저장 중...
🎉 성공적으로 'musique_train.json' 파일에 저장했습니다.


In [ ]:
import tqdm
import
def get_retrieval(retrieve_url: str, queries: List[str], batch_size: int = 64, retry: int = 3) :
    """배치 처리 및 재시도 로직이 포함된 문서 검색 API 호출"""
    results = []
    for i in tqdm.tqdm(range(0, len(queries), batch_size), desc="Retrieving documents"):
        subset = queries[i:i + batch_size]
        for attempt in range(retry):
            try:
                response = requests.post(retrieve_url, json={"queries": subset}, headers={"Content-Type": "application/json"}, timeout=30)
                if response.status_code == 200 and response.json():
                    results.extend(response.json())
                    break
            except requests.exceptions.RequestException as e:
                print(f"Request failed: {e}, attempt {attempt + 1}/{retry}...")
                time.sleep(2)
        else:
            raise ValueError(f"Failed to retrieve queries after {retry} attempts: {subset}")
    return results


In [ ]:
import requests

response = requests.post(retrieve_url, json={"queries": subset}, headers={"Content-Type": "application/json"}, timeout=30)

: 